In [21]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy import stats
from plotly.subplots import make_subplots
import math
import importlib

In [25]:
from fipy import fidata

tickers = [
    {
        "name": "spx",
        "symbol": "^SPX",
        "start": "1920-01-01",
        "period": "1day"
    },
    {
        "name": "vbmfx",
        "symbol": "VBMFX",
        "start": "1920-01-01",
        "period": "1day"
    }
]

fidata.fetch_tickers(tickers)


/home/rennochj/workspace/ml-play/.venv/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning:

The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.

[*********************100%%**********************]  1 of 1 completed
/home/rennochj/workspace/ml-play/.venv/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning:

The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.

[*********************100%%**********************]  1 of 1 completed


In [42]:
from fipy import forecast

importlib.reload(fidata)
importlib.reload(forecast)

YEARS = 30
MONTHS_PER_YEAR = 12
DAYS_PER_MONTH = 21
CURRENT_VALUE = 2_500_000
# EPSILON = 1_000_000 / CURRENT_VALUE
EPSILON = 0 / CURRENT_VALUE
N_PERIODS = YEARS * MONTHS_PER_YEAR * DAYS_PER_MONTH
N_ITERATIONS = 10000
F = 1.025 ** (1.0 / MONTHS_PER_YEAR / DAYS_PER_MONTH )

fig = make_subplots()

for path in ["data/spx.feather", "data/vbmfx.feather"]:
    dist = fidata.load_model(path)
    gamma = forecast.outflow_ratio(dist, EPSILON, F, N_PERIODS, N_ITERATIONS)
    gamma = [g * MONTHS_PER_YEAR * DAYS_PER_MONTH for g in gamma]

    results = np.histogram(gamma, bins=50, density=True)
    results_dist = stats.rv_histogram(results, density=True)

    print(results_dist.ppf(0.2))

    fig.add_trace(
        go.Scatter(
            x=results[1], 
            y=results[0], 
            name=path
        )
    )

fig.show()

0.030683436980822487
(array([ 0.82616393,  3.14729115,  6.78634654, 10.85815447, 12.86455258,
       16.52327854, 16.42492569, 15.73645575, 15.08732695, 14.20215131,
       13.67104593, 11.76300067,  9.36319117,  8.34032155,  7.65185161,
        6.13721774,  4.76027786,  4.17016077,  3.59971425,  2.7932209 ,
        2.26211551,  1.96705697,  1.61298671,  1.4949633 ,  0.94418734,
        0.82616393,  0.60978766,  0.37374082,  0.33439968,  0.35407025,
        0.21637627,  0.27538798,  0.21637627,  0.07868228,  0.01967057,
        0.13769399,  0.05901171,  0.05901171,  0.01967057,  0.03934114,
        0.        ,  0.01967057,  0.01967057,  0.        ,  0.01967057,
        0.        ,  0.        ,  0.        ,  0.        ,  0.01967057]), array([0.00376841, 0.00885266, 0.0139369 , 0.01902115, 0.02410539,
       0.02918964, 0.03427388, 0.03935813, 0.04444237, 0.04952662,
       0.05461086, 0.05969511, 0.06477935, 0.0698636 , 0.07494784,
       0.08003209, 0.08511634, 0.09020058, 0.09528483, 